In [26]:
%pip install psycopg

from urllib.parse import quote_plus
import numpy as np
from pgvector.psycopg import register_vector
import psycopg

# Database connection details
username = "citus"
password = quote_plus("password")  # URL-encode the password
host = "c-postgrescluster-cosmosconf2024.pte3wsxf6htgv3.postgres.cosmos.azure.com"
port = "5432"
dbname = "citus"
DATABASE_URL = f"postgresql://{username}:{password}@{host}:{port}/{dbname}?sslmode=require"

Note: you may need to restart the kernel to use updated packages.


In [27]:
# generate random data
rows = 1000000
dimensions = 128
embeddings = np.random.rand(rows, dimensions)

# enable extension
conn = psycopg.connect(DATABASE_URL, autocommit=True)
conn.execute('CREATE EXTENSION IF NOT EXISTS vector')
register_vector(conn)

# create table
conn.execute('DROP TABLE IF EXISTS vector_documents')
conn.execute(f'CREATE TABLE vector_documents (id bigserial, embedding vector({dimensions}))')

# load data
print(f'Loading {len(embeddings)} rows')
cur = conn.cursor()
with cur.copy('COPY vector_documents (embedding) FROM STDIN WITH (FORMAT BINARY)') as copy:
    # use set_types for binary copy
    # https://www.psycopg.org/psycopg3/docs/basic/copy.html#binary-copy
    copy.set_types(['vector'])

    for i, embedding in enumerate(embeddings):
        # show progress
        if i % 10000 == 0:
            print('.', end='', flush=True)

        copy.write_row([embedding])

        # flush data
        while conn.pgconn.flush() == 1:
            pass

print('\nSuccess!')

Loading 1000000 rows
....................................................................................................
Success!


In [28]:
# create any indexes *after* loading initial data (skipping for this example)
if False:
    print('Creating index')
    conn.execute("SET maintenance_work_mem = '8GB'")
    conn.execute('SET max_parallel_maintenance_workers = 7')
    conn.execute('CREATE INDEX ON items USING hnsw (embedding vector_cosine_ops)')

# update planner statistics for good measure
conn.execute('ANALYZE vector_documents')

<psycopg.Cursor [COMMAND_OK] [IDLE] (host=c-postgrescluster-cosmosconf2024.pte3wsxf6htgv3.postgres.cosmos.azure.com database=citus) at 0x16b3a4b40>